In [ ]:
import cv2
import os
import glob
import time
import pandas as pd
import numpy as np

from skimage.data import astronaut
from skimage.segmentation import slic
from scipy.spatial import Delaunay
from skimage.segmentation import mark_boundaries
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt

from skimage.color import label2rgb
from skimage.transform import resize
from scipy.spatial.distance import cdist

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models

## For Train Folder

In [ ]:
# Selecting dataset and k
dataset_name = 'BUSI-with-GT'  # 'BUSI-with-GT' or  'BUSBRA' 
k = 6  # '2' '4' '6' '8'


patch_dir  = 'Dataset/' + dataset_name + '/data-edge/k' + str(k) + '/'
folder = 'train'

# Usage
folder_path = patch_dir + folder + '/' + 'patch-image/All/' # replace with your folder path
model_path = patch_dir + 'Vgg_model.h5'

attrs = []

def extract_features(model, img_array):
    # img = image.load_img(image_path, target_size=(image_shape[0], image_shape[1]))
    # img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    features = model.predict(img_array, verbose=0)
    return features.flatten()


model_vgg = tf.keras.models.load_model(model_path)

model = Model(model_vgg.input, model_vgg.layers[-2].output)

image_files = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))
images = []
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    feature_vector = extract_features(model, image)
    feature_vector = feature_vector.reshape(1, -1)
    feature_vector = feature_vector.tolist()
    feature_vector = feature_vector[0]

    attrs.append(feature_vector)


    print(image_file)


df_node_attr = pd.DataFrame(data = np.array(attrs))
print("shape of node-attribute dataframe: " + str(df_node_attr.shape))
print("\n--summary of dataframe--\n", df_node_attr.head(50))

sourcepath = patch_dir + 'raw'
def save_dataframe_to_txt(df, filepath):
    df.to_csv(filepath, header=None, index=None, sep=',', mode='w')

save_dataframe_to_txt(df_node_attr, sourcepath + '/train_node_attributes.txt')

## For Test Folder

In [ ]:
# Selecting dataset and k
dataset_name = 'BUSI-with-GT'  # 'BUSI-with-GT' or  'BUSBRA' 
k = 6  # '2' '4' '6' '8'


patch_dir  = 'Dataset/' + dataset_name + '/data-edge/k' + str(k) + '/'
folder = 'test'

# Usage
folder_path = patch_dir + folder + '/' + 'patch-image/All/' # replace with your folder path
model_path = patch_dir + 'Vgg_model.h5'

attrs = []

def extract_features(model, img_array):
    # img = image.load_img(image_path, target_size=(image_shape[0], image_shape[1]))
    # img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    features = model.predict(img_array, verbose=0)
    return features.flatten()



model_vgg = tf.keras.models.load_model(model_path)

model = Model(model_vgg.input, model_vgg.layers[-2].output)


image_files = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))
images = []
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    feature_vector = extract_features(model, image)
    feature_vector = feature_vector.reshape(1, -1)
    feature_vector = feature_vector.tolist()
    feature_vector = feature_vector[0]

    attrs.append(feature_vector)


    print(image_file)


df_node_attr = pd.DataFrame(data = np.array(attrs))
print("shape of node-attribute dataframe: " + str(df_node_attr.shape))
print("\n--summary of dataframe--\n", df_node_attr.head(50))


sourcepath = patch_dir + 'raw'
def save_dataframe_to_txt(df, filepath):
    df.to_csv(filepath, header=None, index=None, sep=',', mode='w')

save_dataframe_to_txt(df_node_attr, sourcepath + '/test_node_attributes.txt')